In [1]:
import os

import pathlib
import tensorflow as tf
import cv2
import argparse
import imutils

In [2]:
args={"model":"/Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection/trainning/13-jan/exported-model-12-jan",
"labels":"/Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection/trainning/maps.pbtxt",
"video":"//Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection/camera/my_future/cervejas_patagonia_brahma.mp4",
"threshold":0.5}


In [3]:
# PROVIDE PATH TO IMAGE DIRECTORY
VIDEO_PATHS = args.get('video')


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = args.get('model')

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = args.get('labels')

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(args.get('threshold'))

In [4]:

# Load the model
# ~~~~~~~~~~~~~~
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print(f"Loading model...{PATH_TO_SAVED_MODEL}", end="")
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print("Done! Took {} seconds".format(elapsed_time))

# Load label map data (for plotting)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS, use_display_name=True
)

Loading model.../Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection/trainning/13-jan/exported-model-12-jan/saved_modelDone! Took 11.326109170913696 seconds


In [5]:

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")  # Suppress Matplotlib warnings


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


print("Running inference for {}... ".format(VIDEO_PATHS), end="")


Running inference for //Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection/camera/my_future/cervejas_patagonia_brahma.mp4... 

In [ ]:
video = cv2.VideoCapture(VIDEO_PATHS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
out = cv2.VideoWriter('~/output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 20.0, size)

while video.isOpened():

    # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    ret, frame = video.read()
    if not ret:
        print("Video ended")
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_expanded = np.expand_dims(frame_rgb, axis=0)
    imH, imW, _ = frame.shape

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(frame)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop("num_detections"))
    detections = {
        key: value[0, :num_detections].numpy() for key, value in detections.items()
    }
    detections["num_detections"] = num_detections

    # detection_classes should be ints.
    detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

    # SET MIN SCORE THRESH TO MINIMUM THRESHOLD FOR DETECTIONS

    detections["detection_classes"] = detections["detection_classes"].astype(np.int64)
    scores = detections["detection_scores"]
    boxes = detections["detection_boxes"]
    classes = detections["detection_classes"]
    count = 0
    for i in range(len(scores)):
        if (scores[i] > MIN_CONF_THRESH) and (scores[i] <= 1.0):
            # increase count
            count += 1
            # Get bounding box coordinates and draw box
            # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
            ymin = int(max(1, (boxes[i][0] * imH)))
            xmin = int(max(1, (boxes[i][1] * imW)))
            ymax = int(min(imH, (boxes[i][2] * imH)))
            xmax = int(min(imW, (boxes[i][3] * imW)))

            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)
            # Draw label
            object_name = category_index[int(classes[i])][
                "name"
            ]  # Look up object name from "labels" array using class index
            label = "%s: %d%%" % (
                object_name,
                int(scores[i] * 100),
            )  # Example: 'person: 72%'
            labelSize, baseLine = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2
            )  # Get font size
            label_ymin = max(
                ymin, labelSize[1] + 10
            )  # Make sure not to draw label too close to top of window
            cv2.rectangle(
                frame,
                (xmin, label_ymin - labelSize[1] - 10),
                (xmin + labelSize[0], label_ymin + baseLine - 10),
                (255, 255, 255),
                cv2.FILLED,
            )  # Draw white box to put label text in
            cv2.putText(
                frame,
                label,
                (xmin, label_ymin - 7),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 0, 0),
                2,
            )  # Draw label text
            print(f"Total Detections : {str(count)} - {label}")

    cv2.putText(
        frame,
        "Total Detections : " + str(count),
        (10, 25),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (70, 235, 52),
        2,
        cv2.LINE_AA,
    )
#     cv2.imshow("Object Detector", frame)
    
    out.write(frame)

    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()
video.release()
out.release()
print("Done")

In [ ]:
!ls